<a href="https://colab.research.google.com/github/BanSangSu/Huggingface-AI-Agents-course/blob/main/unit2/langgraph/agent_with_RAG_tool_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -U rank_bm25 datasets langchain langchain_community langchain_openai langgraph langchain_core
# You can use langchain_huggingface, instead of langchain_openai if you want

In [ ]:
from huggingface_hub import login
login()

## Load and Prepare the Dataset

In [ ]:
import datasets
from langchain_core.documents import Document

guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

docs = [
    Document(
        page_content='\n'.join([
        f"Name: {guest['name']}",
        f"Relation: {guest['relation']}",
        f"Description: {guest['description']}",
        f"Email: {guest['email']}",
    ]),
        metadata={"name": guest["name"]}
    )
    for guest in guest_dataset
]

## Create the Retriever Tool

In [ ]:
from langchain_community.retrievers import BM25Retriever
from langchain.tools import Tool

bm25_retriever = BM25Retriever.from_documents(docs)

def extract_text(query: str) -> str:
    """Retrieves detailed information about gala guests based on their name or relation."""
    results = bm25_retriever.invoke(query)
    if results:
        return "\n\n".join([doc.page_content for doc in results[:3]])
    else:
        return "No matching guest information found."

guest_info_tool = Tool(
    name="guest_info_retriever",
    description="Retrieves detailed information about gala guests based on their name or relation.",
    func=extract_text
)

## Integrate the Tool with Alfred (agent)

In [ ]:
# For alternatives.
import os

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")
os.environ["OPENAI_API_BASE"] = getpass("OpenAI OPENAI BASE URL:")

In [ ]:
from getpass import getpass

model = getpass("Model:")

In [ ]:
# from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

## I've already exceeded my monthly included credits for Inference Providers at Huggingface...
# llm = HuggingFaceEndpoint(
#     repo_id="Qwen/Qwen2.5-Coder-32B-Instruct",
#     # huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
# )

# So I used alternatives.
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=model,
    temperature=0,
)

tools = [guest_info_tool]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition


class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

def assistant(state: AgentState):
    return {
        "messages": [llm_with_tools.invoke(state["messages"])]
    }


builder = StateGraph(AgentState)

builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition
)
builder.add_edge("tools", "assistant")
alfred = builder.compile()

messages = [HumanMessage(content="Tell me about our guest named 'Lady Ada Lovelace'.")]
response = alfred.invoke({"messages": messages})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)